In [43]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config('spark.shuffle.useOldFetchProtocol','true'). \
config("spark.sql.warehouse.dir", f"/user/itv007136/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [44]:
spark

In [3]:
spark.sql("""
select member_id,count(*) as total from itv007136_lending_club.customers
group by member_id
order by total desc
""")

member_id,total
e4c167053d5418230...,5
ad8e5d384dae17e06...,4
76b577467eda5bdbc...,4
3f87585a20f702838...,4
819453be77718d747...,3
c92062bb371842b3d...,3
035bf3d8288d803bd...,3
53789bea7edc660ed...,3
291ca1b09ef11ca3e...,3
4ab6205de571ccb7b...,3


In [4]:
spark.sql("""
select member_id,count(*) as total from itv007136_lending_club.loans_defaulters_delinq
group by member_id
order by total desc
""")

member_id,total
e4c167053d5418230...,3
f1ebb6edb9b07de5f...,2
0b44a10bc749eabb2...,2
db887c1d4cb004ca8...,2
6c2b63ff231e520d4...,2
62394e3f9d063413b...,2
dbe5ec23d01598dbf...,2
c23eb88fed4893030...,2
bbe43331566910d55...,2
a93a44c9d83793451...,2


In [5]:
spark.sql("""
select member_id,count(*) as total from itv007136_lending_club.loans_defaulters_detail_rec_enq
group by member_id
order by total desc
""")

member_id,total
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
3f87585a20f702838...,4
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
3ae415acd6bbfaac1...,3
066ddaa64bee66dff...,3
d9ce4046daf599732...,3
22593a1870543b2db...,3
819453be77718d747...,3


In [45]:
bad_data_customers_df=spark.sql(""" select member_id from
(select member_id,count(*) as total from itv007136_lending_club.customers
group by member_id having total>1)
""")

In [46]:
bad_data_customers_df.count()

3157

In [47]:
bad_data_loans_defaulters_delinq_df=spark.sql(""" select member_id from(
select member_id,count(*) as total from itv007136_lending_club.loans_defaulters_delinq
group by member_id
having total>1)
""")

In [48]:
bad_data_loans_defaulters_delinq_df.count()

173

In [49]:
bad_data_loans_defaulters_detail_rec_enq_df=spark.sql("""select * from
(select member_id,count(*) as total from itv007136_lending_club.loans_defaulters_detail_rec_enq
group by member_id
having total>1)
""")

In [50]:
bad_data_loans_defaulters_detail_rec_enq_df.count()

3189

In [23]:
bad_data_customers_df.repartition(1).write.format("csv").option("header","true").mode("overwrite").option("path","/user/itv007136/lendingclubproject/bad/bad_data_customers").save()

In [24]:
bad_data_loans_defaulters_delinq_df.repartition(1).write.format("csv").option("header","true").mode("overwrite").option("path","/user/itv007136/lendingclubproject/bad/bad_data_loans_defaulters_delinq").save()

In [25]:
bad_data_loans_defaulters_detail_rec_enq_df.repartition(1).write.format("csv").option("header","true").mode("overwrite").option("path","/user/itv007136/lendingclubproject/bad/bad_data_loans_defaulters_detail_enq").save()

In [51]:
bad_customers_data_df=bad_data_customers_df.select("member_id"). \
union(bad_data_loans_defaulters_delinq_df.select("member_id")). \
union(bad_data_loans_defaulters_detail_rec_enq_df.select("member_id"))

In [52]:
bad_customers_data_final_df=bad_customers_data_df.distinct()

In [53]:
bad_customers_data_final_df.count()

3189

In [30]:
bad_customers_data_final_df.repartition(1).write.format("csv").option("header","true").mode("overwrite").option("path","/user/itv007136/lendingclubproject/bad/bad_customer_data_final").save()

In [54]:
bad_customers_data_final_df.createOrReplaceTempView("bad_data_customer")

In [55]:
customers_df=spark.sql("""
select * from itv007136_lending_club.customers
where member_id not in (select member_id from bad_data_customer)
""")

In [33]:
customers_df.write.format("parquet").mode("overwrite").option("path","/user/itv007136/lendingclubproject/cleaned_new/customers_parquet").save()

In [56]:
loan_defaulters_delinq_df=spark.sql("""
select * from itv007136_lending_club.loans_defaulters_delinq
where member_id not in (select member_id from bad_data_customer)
""")

In [35]:
loan_defaulters_delinq_df.write.format("parquet").mode("overwrite").option("path","/user/itv007136/lendingclubproject/cleaned_new/loans_defaulters_delinq_parquet").save()

In [57]:
loan_defaulters_detail_rec_enq_df=spark.sql("""
select * from itv007136_lending_club.loans_defaulters_detail_rec_enq
where member_id not in (select member_id from bad_data_customer)
""")

In [37]:
loan_defaulters_detail_rec_enq_df.write.format("parquet").mode("overwrite").option("path","/user/itv007136/lendingclubproject/cleaned_new/loans_defaulters_detail_rec_enq_parquet").save()

In [58]:
customers_df

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
de8d40bb549f3e676...,Slot Club Lead,10,RENT,42000.0,CO,800xx,USA,A,A5,Source Verified,19200.0,Individual,null,null,2023-09-15 17:18:...
cb2bfe6ba8d8040ea...,Development Manager,1,MORTGAGE,120000.0,NJ,080xx,USA,B,B2,Verified,521247.0,Individual,null,null,2023-09-15 17:18:...
fd6cd0f257e376951...,Chef/Baker,5,RENT,25000.0,GA,310xx,USA,E,E2,Source Verified,18200.0,Individual,null,null,2023-09-15 17:18:...
46a7d1486ffcd4024...,Service Manager,3,MORTGAGE,122000.0,NY,113xx,USA,C,C3,Verified,232468.0,Individual,null,null,2023-09-15 17:18:...
ec70891c26a58aad8...,Maintenance Manager,1,RENT,35000.0,CO,804xx,USA,C,C1,Verified,16200.0,Individual,null,null,2023-09-15 17:18:...
8b5e4553da13c2336...,Driver,10,RENT,65000.0,NY,112xx,USA,C,C2,Verified,15200.0,Individual,null,null,2023-09-15 17:18:...
ea940e76460ba9551...,Carpenter,2,MORTGAGE,72000.0,IL,622xx,USA,B,B3,Source Verified,75875.0,Individual,null,null,2023-09-15 17:18:...
1bff1ca3a1f99b4d7...,Owner,3,RENT,103000.0,NC,282xx,USA,D,D2,Source Verified,18700.0,Individual,null,null,2023-09-15 17:18:...
78caf872bd06843ef...,Rural Carrier,10,OWN,70000.0,CA,932xx,USA,B,B4,Source Verified,71235.0,Individual,null,null,2023-09-15 17:18:...
8c0e5cf855a938944...,NotAsked,10,RENT,87600.0,CO,802xx,USA,B,B2,Source Verified,84826.0,Individual,null,null,2023-09-15 17:18:...


In [39]:
spark.sql("""
CREATE EXTERNAL TABLE itv007136_lending_club.customers_new(
member_id string,emp_title string,emp_length int,home_ownership string,annual_income float,
address_state string,address_zipcode string,address_country string,grade string,sub_grade string,
verification_status string,total_high_credit_limit float,application_type string,
join_annual_income float,verification_status_joint string,ingest_date timestamp
) stored as parquet location '/public/trendytech/lendingclubproject/cleaned_new/customers_parquet'
""")

""


In [59]:
loan_defaulters_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
9cb79aa7323e81be1...,2,0.0,0
aac68850fdac09fd0...,1,0.0,0
c89986155a070db2e...,1,0.0,15
4e1c30a5dfe9f1e20...,0,0.0,34
1eef79a0e79b72c7a...,1,0.0,0
ec1953dba2cfb89ad...,2,0.0,0
3712c9da85e54b7b1...,1,0.0,0
030e5c9c411c936c5...,2,0.0,0
252736f8401b6b046...,2,0.0,0
2d73c65a7b7d1a351...,1,0.0,0


In [60]:
spark.sql("""
CREATE EXTERNAL TABLE itv007136_lending_club.loan_defaulters_delinq_new(
member_id string,delinq_2yrs integer,delinq_amnt float,mths_since_last_delinq integer
) stored as parquet location '/public/trendytech/lendingclubproject/cleaned_new/loan_defaulters_delinq_parquet'
""")

""


In [61]:
loan_defaulters_detail_rec_enq_df

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
9cb79aa7323e81be1...,0,0,0
0dd2bbc517e3c8f9e...,1,1,3
458458599d3df3bfc...,1,1,1
05ea141ec28b5c7f7...,0,0,0
aac68850fdac09fd0...,0,0,0
3a423e4589e89f429...,0,0,0
f1efcf7dfbfef21be...,0,0,1
c89986155a070db2e...,0,0,1
118dc629b6e134419...,0,0,0
a86fa4b7493708333...,0,0,0


In [65]:
spark.sql("""
CREATE EXTERNAL TABLE itv007136_lending_club.loan_defaulters_detail_rec_enq_new(
member_id string,pub_rec integer,pub_rec_bankruptcies integer,inq_last_6mths integer
) stored as parquet location '/public/trendytech/lendingclubproject/cleaned_new/loan_defaulters_detail_rec_enq_parquet'
""")

""
